<a href="https://colab.research.google.com/github/gadilshina-ve/SoftwareTools_DA/blob/main/LW_01/LW_01_Gadilshina_13_09_2024_SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install requests beautifulsoup4 pymongo pandas matplotlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
url = 'https://greenway.icnet.ru/cars-sales-actual-usa.html#null'
response = requests.get(url)
page_content = response.content

In [4]:
# Проверяем успешность запроса
if response.status_code == 200:
    # Получаем HTML-код страницы
    html_content = response.text

In [5]:
soup = BeautifulSoup(page_content, 'html.parser')

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [6]:
print(soup.prettify())  # Выводим HTML-код страницы для анализа

<!DOCTYPE html>
<html lang="ru">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="Продажи автомобилей в США в 2024 году. Статистика продажи автомобилей по годам и месяцам" name="description"/>
  <meta content="Статистика и рейтинги продаж автомобилей в США в 2024 году" name="keywords"/>
  <title>
   Полная статистика продаж автомобилей в США в 2024 году
  </title>
  <link href="a-style.css" rel="stylesheet"/>
  <script src="jquery-3.5.1.min.js">
  </script>
  <script async="" src="https://yastatic.net/share2/share.js">
  </script>
  <script src="sorttable.js">
  </script>
  <!--Скрипт для сортировки таблиц -->
  <script src="https://www.google.com/jsapi">
  </script>
  <!--Скрипт для графиков -->
  <script>
   window.yaContextCb=window.yaContextCb||[]
  </script>
  <!-- Yandex.RTB -->
  <script async="" src="https://yandex.ru/ads/system/context.js">
  </script>
  <sc

In [8]:
table = soup.find('table', {'id': 'unique_id'})  # Ищем таблицу с id='unique_id'
rows = table.find_all('tr')  # Находим все строки таблицы

In [9]:
data = []
for row in rows[1:]:  
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append(cols)

# Создаем DataFrame
df = pd.DataFrame(data, columns=['Rank', 'Brand', 'Sales_2024', 'Sales_2023', 'Change_percent_2024',
                                 'Brand_2', 'Cumulative_2024', 'Cumulative_2023', 'Cumulative_Change_percent'])

# Заменяем '-' на NaN, чтобы избежать ошибок при преобразовании
df.replace('-', np.nan, inplace=True)

df['Sales_2024'] = df['Sales_2024'].str.replace(',', '.').astype(float)
df['Sales_2023'] = df['Sales_2023'].str.replace(',', '.').astype(float)
df['Change_percent_2024'] = df['Change_percent_2024'].str.replace(',', '.').astype(float)
df['Cumulative_2024'] = df['Cumulative_2024'].str.replace(',', '.').astype(float)
df['Cumulative_2023'] = df['Cumulative_2023'].str.replace(',', '.').astype(float)
df['Cumulative_Change_percent'] = df['Cumulative_Change_percent'].str.replace(',', '.').astype(float)

In [10]:
print(df)

  Rank      Brand  Sales_2024  Sales_2023  Change_percent_2024    Brand_2  \
0    1     Toyota       198.0       194.0                  2.4     Toyota   
1    2       Ford       176.0       160.0                 10.1       Ford   
2    3  Chevrolet       151.0       152.0                 -0.7  Chevrolet   
3    4      Honda       140.0       112.0                 25.0      Honda   
4    5    Hyundai        87.0        71.0                 21.2     Nissan   
5    6        Kia        75.0        72.0                  4.3    Hyundai   
6    7     Nissan        71.0        73.0                 -1.4        Kia   
7    8     Subaru        63.0        56.0                 11.8     Subaru   
8    9      Tesla        53.0        58.0                 -9.3      Tesla   
9   10      Mazda        41.0        30.0                 36.7      Mazda   

   Cumulative_2024  Cumulative_2023  Cumulative_Change_percent  
0           1567.0           1425.0                       10.0  
1           1380.0    

In [11]:
conn = sqlite3.connect('financial_data.db')
df.to_sql('car_sales', conn, if_exists='replace', index=False)

10

In [14]:
zapros = "SELECT * FROM car_sales WHERE Brand = 'Tesla'"
df_sqlite1 = pd.read_sql(zapros, conn)
print(df_sqlite1)

  Rank  Brand  Sales_2024  Sales_2023  Change_percent_2024 Brand_2  \
0    9  Tesla        53.0        58.0                 -9.3   Tesla   

   Cumulative_2024  Cumulative_2023  Cumulative_Change_percent  
0            394.0            453.0                      -13.0  


In [15]:
zapros2 = "SELECT * FROM car_sales WHERE Cumulative_Change_percent > 70"
df_sqlite2 = pd.read_sql(zapros2, conn)
print(df_sqlite2)

Empty DataFrame
Columns: [Rank, Brand, Sales_2024, Sales_2023, Change_percent_2024, Brand_2, Cumulative_2024, Cumulative_2023, Cumulative_Change_percent]
Index: []


In [16]:
zapros3 = "SELECT * FROM car_sales WHERE Sales_2024 > Sales_2023"
df_sqlite3 = pd.read_sql(zapros3, conn)
print(df_sqlite3)

  Rank    Brand  Sales_2024  Sales_2023  Change_percent_2024  Brand_2  \
0    1   Toyota       198.0       194.0                  2.4   Toyota   
1    2     Ford       176.0       160.0                 10.1     Ford   
2    4    Honda       140.0       112.0                 25.0    Honda   
3    5  Hyundai        87.0        71.0                 21.2   Nissan   
4    6      Kia        75.0        72.0                  4.3  Hyundai   
5    8   Subaru        63.0        56.0                 11.8   Subaru   
6   10    Mazda        41.0        30.0                 36.7    Mazda   

   Cumulative_2024  Cumulative_2023  Cumulative_Change_percent  
0           1567.0           1425.0                       10.0  
1           1380.0           1333.0                        3.6  
2            951.0            855.0                       11.2  
3            636.0            624.0                        1.9  
4            593.0            570.0                        4.1  
5            437.0       

In [18]:
zapros4 = "SELECT * FROM car_sales WHERE Sales_2024 > (SELECT AVG(Sales_2023) FROM car_sales)"
df_sqlite4 = pd.read_sql(zapros4, conn)
print(df_sqlite4)

  Rank      Brand  Sales_2024  Sales_2023  Change_percent_2024    Brand_2  \
0    1     Toyota       198.0       194.0                  2.4     Toyota   
1    2       Ford       176.0       160.0                 10.1       Ford   
2    3  Chevrolet       151.0       152.0                 -0.7  Chevrolet   
3    4      Honda       140.0       112.0                 25.0      Honda   

   Cumulative_2024  Cumulative_2023  Cumulative_Change_percent  
0           1567.0           1425.0                       10.0  
1           1380.0           1333.0                        3.6  
2           1176.0           1190.0                       -1.2  
3            951.0            855.0                       11.2  
